## Preprocessing for Local

In [1]:
import pandas as pd
import re
from tqdm import tqdm

# ---------- Helper function: aggregate_posts ----------
def aggregate_posts(df: pd.DataFrame) -> pd.DataFrame:
    # Step 1: convert post_time to datetime
    df["post_time"] = pd.to_datetime(df["post_time"].str.split(",").str[0], format="%d/%m/%Y", errors="coerce")

    # Step 2: create formatted post string
    def format_post(row):
        if row["post_type"] == "reply":
            return f"- Reply to content: {row['reply_to_post']} | Replied content: {row['content']}"
        else:
            return f"- {row['content']}"
    
    df["post_string"] = df.apply(format_post, axis=1)
    df = df.rename(columns={"post_time": "date"})
    
    # Step 3: aggregate data grouped by date and topic_title
    grouped = (
        df.groupby(["date", "topic_title"], as_index=False)
          .agg(posts=("post_string", lambda x: "\n".join(x)))
    )

    # Step 4: filter out posts before 2020-01-01
    grouped = grouped[grouped["date"] > "2020-01-01"]

    return grouped


# ---------- Main preprocessing function ----------
def f319_data_preprocessing(file_paths, output_path="f319_data/aggregated_f319_data/aggregated_f319_posts.csv"):
    """
    Preprocess all F319 CSV files and aggregate posts per date and topic.

    Args:
        file_paths: list of local CSV file paths
        output_path: output file name (default 'aggregated_f319_posts.csv')
    """
    all_dfs = []

    for path in tqdm(file_paths, desc="Processing files"):
        # Read CSV
        df = pd.read_csv(path, header=0, encoding="utf-8", escapechar="\\", quoting=1, on_bad_lines="skip")
        
        # Drop rows with nulls in essential columns
        df = df.dropna(subset=["topic_title", "post_time", "content"])
        
        # Detect replies and original posts
        reply_pattern = r"^(.*?) đã viết:\n↑\n"
        reply_extract = r"^.*? đã viết:\n↑\n(.*?)(?:\nXem tất cả)?\n"
        
        df["post_type"] = df["content"].apply(lambda x: "reply" if re.search(reply_pattern, x) else "original")
        df["reply_to_post"] = df.apply(
            lambda row: re.search(reply_extract, row["content"]).group(1)
                        if row["post_type"] == "reply" and re.search(reply_extract, row["content"])
                        else "",
            axis=1
        )
        df["content"] = df.apply(
            lambda row: re.sub(reply_extract, "", row["content"]) if row["post_type"] == "reply" else row["content"],
            axis=1
        )
        
        # Aggregate
        agg_df = aggregate_posts(df)
        all_dfs.append(agg_df)

    # Combine all aggregated DataFrames
    final_df = pd.concat(all_dfs, ignore_index=True)
    print(f"Total aggregated entries before final grouping: {len(final_df)}")

    # Save final CSV
    final_df.to_csv(output_path, index=False, encoding="utf-8")
    print(f"✅ Aggregated data saved to: {output_path}")

    return final_df



In [2]:
if __name__ == "__main__":
    file_paths = [f"f319_data/posts_{i:04d}.csv" for i in range(1, 514)]
    final_df = f319_data_preprocessing(file_paths)


Processing files: 100%|██████████| 513/513 [04:21<00:00,  1.96it/s]


Total aggregated entries before final grouping: 518299
✅ Aggregated data saved to: f319_data/aggregated_f319_data/aggregated_f319_posts.csv


## Load aggregated file for summarization

In [1]:
import pandas as pd
df = pd.read_csv("f319_data/aggregated_f319_data/aggregated_f319_posts.csv")


In [2]:
df

,date,topic_title,posts
0,2020-01-03,Phân tích kỹ thuật (TA) - (chỉ chia sẻ kiến th...,- Reply to content: Bác nào giỏi TA cho cái bi...
1,2020-01-14,Phân tích kỹ thuật (TA) - (chỉ chia sẻ kiến th...,- phân tích kỹ thuật nên lựa mã mới phân tích....
2,2020-01-19,Phân tích kỹ thuật (TA) - (chỉ chia sẻ kiến th...,- bác\n@nano9\nxem giúp em ảnh ở trang 1 đến 1...
3,2020-02-13,Phân tích kỹ thuật (TA) - (chỉ chia sẻ kiến th...,- bác nào muốn nghiên cứu phân tích kỹ thuật t...
4,2020-02-18,Phân tích kỹ thuật (TA) - (chỉ chia sẻ kiến th...,"- Sau sự kiện Nam tiên sinh, Giám đốc đào tạo ..."
...,...,...,...
518294,2020-01-10,Triệu tập Hội Đồng Gió Mát - Tập 03,"- Reply to content: Cuối năm rầu, nhà bao việc..."
518295,2020-01-16,Triệu tập Hội Đồng Gió Mát - Tập 03,"- Vcc rồi, a sốc not official rồi\n- Reply to ..."
518296,2020-01-17,Triệu tập Hội Đồng Gió Mát - Tập 03,- Reply to content: Ae ib Sđt tui giao hlu nhé...
518297,2020-01-20,Triệu tập Hội Đồng Gió Mát - Tập 03,- Đâu rồi hội đồng của những ngày xưa cũ


In [3]:
df['topic_title'].nunique()

107100

In [23]:
# search for topic_title with keyword "" 
df[df['topic_title'].str.contains("VPBS", case=False, na=False)]


,date,topic_title,posts
1919,2025-10-20,VPB bay cao cùng IPO VPBS,- Chúng tôi khuyến nghị mua đối cổ phiếu VPB với giá mục tiêu là 37.000 đồng/cp. Giá mục tiêu củ...
1963,2025-10-23,VPB bay cao cùng IPO VPBS,- VPB – BCPT – Tái định giá với thương vụ IPO VPBankS\nLợi nhuận trước thuế của VPB dự kiến tăng...
2003,2025-10-25,VPB bay cao cùng IPO VPBS,"- VPB họ đang có kế hoạch bán 5% vốn cho một ngân hàng của Nhật Bản, giá đâu đó tầm 40x, đây là ..."
2043,2025-10-26,VPB bay cao cùng IPO VPBS,- Reply to content: Tôi đang chờ VPB về tầm 27 là gom dần vì VPB là NH tư nhân có chiến lược tốt...
18350,2025-10-16,VPBS – Có nên mua khi IPO 33.9k/cp,- Hiện VPBS đã công bố giá IPO sẽ là 33.9k với lượng cổ phiếu dự kiến chào bán ra lần đầu là kho...
18445,2025-10-22,VPBS – Có nên mua khi IPO 33.9k/cp,"- Mua IPO chính ra lại an toàn!\nNé được mấy cú hập hầm mà lại còn lãi chắc, VPX giá thấp nhất m..."
18477,2025-10-26,VPBS – Có nên mua khi IPO 33.9k/cp,"- Reply to content: Đắt VPX, né đc tháng này TT bất ổn thì ăn chắc | Replied content: Con hàng q..."
28899,2025-10-09,"Tcbs IPO thì Tcb tăng 10%, Vpbs IPO thì Vpb...?",- Ngày 19/8/2025 TCBS chính thức IPO\nTCB là công ty mẹ của TCBS. Ngay lập tức ngày 19/8 TCB bật...
28909,2025-10-10,"Tcbs IPO thì Tcb tăng 10%, Vpbs IPO thì Vpb...?","- Nhịp này chắc lên 35-36, rồi test lại, chưa lên dc 38 ngay đâu\nAdamWord đã viết:\n↑\n38 nha c..."
28968,2025-10-14,"Tcbs IPO thì Tcb tăng 10%, Vpbs IPO thì Vpb...?","- Nay khối ngoại mua ròng 4 triệu, dòng tiền lớn bắt đầu nhập cuộc\n- tcbs ngáo giá à"
